In [17]:
IMG_PATH = '../recursos/images/recibos/recibo007.png'

# Protótipo com Open AI

In [29]:
import base64
import os
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.tools import CopyFileTool
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

import PIL.Image
from IPython.display import display

load_dotenv()

class ProcessImage:
        
    def __init__(self):
        openai_api_key = os.getenv("OPENAI_API_KEY")
        self.llm = ChatOpenAI(
            model='gpt-4o-mini',
            temperature=0,
            openai_api_key=openai_api_key
        )

    def openai_analysis(self, image_path):
        with open(image_path, "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode("UTF-8")
            
        prompt = ChatPromptTemplate.from_messages(
            messages=[
                HumanMessage(content="""Dada a imagem de uma nota fiscal de compra, extraia os dados importantes dela e retorne no seguinte formato:
                        - Empresa : nome da empresa que emitiu o recibo
                        - Data: a data que foi feita a compra
                        - CNPJ: cnpj da empresa que emitiu o recibo, sem pontuação
                        - Tipo: tipo da despesa baseada nos itens comprados, escolha uma dentre as seguintes categorias:
                                    - Alimentação
                                    - Saúde
                                    - Mercado
                                    - Compras
                                    - Transporte
                        - Itens: 
                            - Descrição: nome ou descrição da mercadoria
                            - Valor Unitário: valor unitário da mercadoria
                            - Quantidade: quantidade da mercadoria
                            - Valor Total: valor total da mercadoria
                        - Valor Pago: valor pago
                        Retorne os dados no formato JSON com nomes dos campos em snake_case.
                        Retorne apenas o json e nenhum outro texto.
                        Retorne apenas dados que existam na imagem, não invente informações, caso não consiga extrair informações, deixe o campo vazio.
                    {text}
                """),
                 HumanMessagePromptTemplate.from_template(
                    template=[
                        {"type": "image_url", "image_url": {"url": "{image_url}"}},
                    ]
                ),
                MessagesPlaceholder("agent_scratchpad"),
            ]
        )
        
        tools = [CopyFileTool()]
        agent = create_openai_tools_agent(self.llm, tools, prompt)
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
        )
        result = agent_executor.invoke({"image_url": f"data:image/jpeg;base64,{base64_image}", "agent_scratchpad": []})
        response = result['output'] 
    
        print(response)
        return response

dados = ProcessImage().openai_analysis(IMG_PATH)

```json
{
    "empresa": "FURLAN IMPORTS",
    "data": "21/02/2025",
    "cnpj": "16570480000166",
    "tipo": "Compras",
    "itens": [
        {
            "descricao": "TECLADO GAMER RAZER CYNOSA CHROMA",
            "valor_unitario": "499.99",
            "quantidade": "1",
            "valor_total": "499.99"
        }
    ],
    "valor_pago": "399.99"
}
```


# Protótipo com Google Gemini

In [28]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os
import PIL.Image
from IPython.display import display

load_dotenv()

def generate(image_path):
    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    image = PIL.Image.open(image_path)
    text1 = """Dada a imagem de uma nota fiscal de compra, extraia os dados importantes dela e retorne no seguinte formato:
    - Empresa : nome da empresa que emitiu o recibo
    - Data: a data que foi feita a compra
    - CNPJ: cnpj da empresa que emitiu o recibo, sem pontuação
    - Tipo: tipo da despesa baseada nos itens comprados, escolha uma dentre as seguintes categorias:
                - Alimentação
                - Saúde
                - Mercado
                - Compras
                - Transporte
    - Itens: 
        - Descrição: nome ou descrição da mercadoria
        - Valor Unitário: valor unitário da mercadoria
        - Quantidade: quantidade da mercadoria
        - Valor Total: valor total da mercadoria
    - Valor Pago: valor pago
    Retorne os dados no formato JSON com nomes dos campos em snake_case.
    Retorne apenas o json e nenhum outro texto.
    Retorne apenas dados que existam na imagem, não invente informações, caso não consiga extrair informações, deixe o campo vazio.
    """

    model = "gemini-2.0-flash-001"
    generate_content_config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 0.95,
        max_output_tokens = 8192,
        response_modalities = ["TEXT"],
    )
    response  = client.models.generate_content(
        model = model,
        contents = [text1, image],
        config = generate_content_config,
    )
    print(response.text)

generate(IMG_PATH)


```json
{
    "empresa": "FURLAN IMPORTS",
    "data": "21/02/2025 14:33.47",
    "cnpj": "16570480000166",
    "tipo": "Compras",
    "itens": [
        {
            "descrição": "TECLADO GAMER RAZER CYNOSA CHROMA",
            "valor_unitario": "499.99",
            "quantidade": "1.00",
            "valor_total": "499.99"
        }
    ],
    "valor_pago": "399,99"
}
```


# Tesseract com o tratamento de Imagem + Open AI pro texto

In [30]:
import numpy as np
import cv2
import imutils
import pytesseract
from matplotlib import pyplot as plt

def encontrar_contornos(img):
  conts = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  conts = imutils.grab_contours(conts)
  conts = sorted(conts, key = cv2.contourArea, reverse = True)[:6]
  return conts


def ordenar_pontos(pontos):
  pontos = pontos.reshape((4,2))
  pontos_novos = np.zeros((4, 1, 2), dtype=np.int32)

  add = pontos.sum(1)
  pontos_novos[0] = pontos[np.argmin(add)]
  pontos_novos[2] = pontos[np.argmax(add)]

  dif = np.diff(pontos, axis = 1)
  pontos_novos[1] = pontos[np.argmin(dif)]
  pontos_novos[3] = pontos[np.argmax(dif)]

  return pontos_novos


def transform_imagem(nome_imagem):
    img = cv2.imread(nome_imagem)
    original = img.copy()
    (H, W) = img.shape[:2]

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    edged = cv2.Canny(blur, 60, 160)
    conts = encontrar_contornos(edged.copy())
    for c in conts:
        peri = cv2.arcLength(c, True)
        aprox = cv2.approxPolyDP(c, 0.02 * peri, True)

        if len(aprox) == 4:
            maior = aprox
            break

    cv2.drawContours(img, maior, -1, (120, 255, 0), 28)
    cv2.drawContours(img, [maior], -1, (120, 255, 0), 2)

    pontosMaior = ordenar_pontos(maior)
    pts1 = np.float32(pontosMaior)
    pts2 = np.float32([[0, 0], [W, 0], [W, H], [0, H]])

    matriz = cv2.getPerspectiveTransform(pts1, pts2)
    transform = cv2.warpPerspective(original, matriz, (W, H))
    
    img_process = cv2.cvtColor(transform, cv2.COLOR_BGR2GRAY)
    img_process = cv2.adaptiveThreshold(img_process, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 9)
    return img_process

img_final = transform_imagem(IMG_PATH)
 
config_tesseract = "--tessdata-dir tessdata"
texto = pytesseract.image_to_string(img_final, lang="por", config=config_tesseract)

img = cv2.imread(IMG_PATH)
texto = pytesseract.image_to_string(img, lang="por", config=config_tesseract)
print(texto)


prompt = ChatPromptTemplate.from_messages(
    messages=[
        HumanMessage(content=f"""Dado um texto extraido de uma nota fiscal de compra, extraia os dados importantes dela e retorne no seguinte formato:
                - Empresa : nome da empresa que emitiu o recibo
                - Data: a data que foi feita a compra
                - CNPJ: cnpj da empresa que emitiu o recibo, sem pontuação
                - Tipo: tipo da despesa baseada nos itens comprados, escolha uma dentre as seguintes categorias:
                            - Alimentação
                            - Saúde
                            - Mercado
                            - Compras
                            - Transporte
                - Itens: 
                    - Descrição: nome ou descrição da mercadoria
                    - Valor Unitário: valor unitário da mercadoria
                    - Quantidade: quantidade da mercadoria
                    - Valor Total: valor total da mercadoria
                - Valor Pago: valor pago
                Retorne os dados no formato JSON com nomes dos campos em snake_case. 
                Retorne apenas o json e nenhum outro texto.
                Retorne apenas dados que existam na imagem, não invente informações, caso não consiga extrair informações, deixe o campo vazio.
            {texto}
        """),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)
agent = create_openai_tools_agent(llm, [], prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=[],
)
result = agent_executor.invoke({"agent_scratchpad": []})
response = result['output'] 

print(response)




FURLAN IMPORTS
CNPJ: 16.570.480/000 1:66 1E: 626447294112
AVENIDA INDUSTHKÍAL, 600 - CAMPESTRE
SANTO ANDRE - SP 09080-501
Fone: (11) 23243610 Celular (11) 91300-3250
E-mail f imports Ogmail.com
e PORTS OFICIAiL
issão: 21/02/2025 14:33.47

Identificação do destinatário

   
 
 
 

Nº Nota: 7025

Cliente LUCAS ”
Endere;:
Compl,;
Bairro
Cidade SANTO ANDRE
VE 367000556 0B
Ú

 

cod. Descrição Qide Unit, Desc. Total
MNO0NARIDR 2A3A TE ) GAMER RAA CYN HROMA

UN 1,00 X 49999 000 J'-y.

   
 

QTDE. TOTAL DEITENS à a
TOTAL BRUTO )

TOTAL DESCONTO R;?:Z:Z
TOTAL LÍQUIDO 399,99
FORMA DE PAGAMENTO VALOR PAGO
CARTÃO CRÉDITO 399,959

EUZA FERKREI!

Vendedo
Obs:

TERMO DE GARANTIA S j
OS PRODUÍTOS TEM GARANTIA 1=2" DE 5 MESES CONTAD«
PARTIR DA DATA DE COMPRA, NOº "ªª“g*““ªmhw.mu
" NTEOU ENVIAR AO RE Pp/
E A FABRI IR AO REPA
ROCARO PRODUTO COM “ceviço sellRES T ADO

OM AFABRICANTE pubtnª_;(" >l ' DIAS VISTO QUE CADA CA
DENTRO DO PRAZO LEGAL DE À'* , x DECOMPRA visTO
DIFERENTE, NÃO REALIZAMOS DESP' " o